a step by step demonstration for scripts/01.create_clean_scene.py

In [ ]:
import os
import time
import pickle
import yaml

import numpy as np

import scene_point_etk.argoverse2 as argoverse2
import scene_point_etk.visualization as visualization

import py_utils.pcd as pcd

In [ ]:
with open("../scripts/configs/overall_configs.yaml", "r") as fd:
    configs = yaml.safe_load(fd)
list(configs.keys())

In [ ]:
log_ids = argoverse2.list_log_ids()
log_ids[:3]

In [ ]:
log_id = log_ids[0]
log_id

---
SweepSequence: a class to manage all measurement from lidar

In [ ]:
sweep_seq = argoverse2.SweepSequence(log_id, coordinate="map")
sweep_seq, sweep_seq.sweeps[:3]

---
removing dynamic objects from `sweep_seq`

In [ ]:
origin_annots = argoverse2.Annotations(log_id)
origin_annots

In [ ]:
dynamic_categories = configs["dynamic_objects_categories"]
dynamic_categories

In [ ]:
dynamic_annots = origin_annots.filter_by_category(dynamic_categories)
dynamic_annots

In [ ]:
s = time.time()
mov_sweep_seq = sweep_seq.filtered_by_annotations(
    dynamic_annots,
    bbox_margin=configs["bbox_params"]["bbox_margin"],
    time_margin=configs["bbox_params"]["time_margin"],
)

print(f"Time taken to filter sweeps: {time.time() - s:.2f} seconds")
mov_sweep_seq

In [ ]:
static_sweep_seq = sweep_seq - mov_sweep_seq

---
visualize SweepSequence and Annotations

In [ ]:
# view before and after filtering

visualization.plot_mulitple_pcds(
    (sweep_seq.xyz, sweep_seq.intensity),
    (static_sweep_seq.xyz, static_sweep_seq.intensity),
)

In [ ]:
# view annotations and sweeps

bboxes = dynamic_annots.bounding_box_vertices(
    margin=configs["bbox_params"]["bbox_margin"]
)

visualization.plot_multiple_pcds_with_bboxes(
    (sweep_seq.xyz, sweep_seq.intensity),
    (static_sweep_seq.xyz, static_sweep_seq.intensity),
    bboxes=bboxes,
    bbox_eps=0.5,
)

---
voxelize static scene

In [ ]:
# target.pcd
gt_pcd, details = static_sweep_seq.export_to_voxel_grid(
    voxel_size=configs["pcd_params"]["voxel_size"],
    return_details=True,
)

# details.pkl
gt_details = {
    "annotations": dynamic_annots,
    "sweeps": static_sweep_seq.sweeps,
    "moving_objects_categories": configs["dynamic_objects_categories"],
    "bbox_margin": configs["bbox_params"]["bbox_margin"],
    "time_margin": configs["bbox_params"]["time_margin"],
    "voxel_size": configs["pcd_params"]["voxel_size"],
}

num_bytes = len(pickle.dumps(gt_details))

print(f"Size of details.pkl: {num_bytes / 1024:.2f} KB")

In [ ]:
# recover sweeps to sweep sequence
argoverse2.SweepSequence.from_sweeps(gt_details["sweep_seq"])

In [ ]:
# visualize the voxel grid
print(gt_pcd, "\n\n", gt_pcd.dtype)

xyz = np.vstack([gt_pcd["x"], gt_pcd["y"], gt_pcd["z"]]).T
intensity = gt_pcd["intensity"]

v = visualization.plot_mulitple_pcds(
    (static_sweep_seq.xyz, static_sweep_seq.intensity), (xyz, intensity)
)
v.set(point_size=0.01)